# 1. Dateien Herunterladen und Laden

In [1]:
import os
import tarfile
from six.moves import urllib
import pandas as pd

DOWNLOAD_ROOT = 'https://github.com/ageron/handson-ml/raw/master/'
HOUSING_PATH = 'datasets/housing/'
HOUSING_URL = DOWNLOAD_ROOT + HOUSING_PATH + 'housing.tgz'

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    """Daten herunterladen und entpacken"""
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
        pass
    tgz_path = os.path.join(housing_path, 'housing.tgz')
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    pass

def load_housing_data(housing_path=HOUSING_PATH):
    """CSV als Pandas DataFrame laden."""
    csv_path = os.path.join(housing_path, 'housing.csv')
    return pd.read_csv(csv_path)

In [2]:
# Daten herunterladen
fetch_housing_data()

In [3]:
# Daten laden
housing = load_housing_data()

# 2. Test und Trainingsdaten trennen

In [4]:
import numpy as np

In [5]:
# das einkommen in kategorien einteilen, damit man einen repräsentativen Test/Trainingsdatensatz erhält
#teile die einkommen in kategorien ein. erst geteilt durch 1.5 um die Anzahl kategorien zu begrenzen
housing['income_cat'] = np.ceil(housing['median_income'] / 1.5)
# ersetze alle über 5 mit 5
housing['income_cat'].where(housing['income_cat'] < 5, 5.0, inplace=True)

In [6]:
from sklearn.model_selection import StratifiedShuffleSplit

In [7]:
# spliter konfigurieren
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
# eine stratifizierte strichprobe als testdaten generieren lassen.
for train_index, test_index in split.split(housing, housing['income_cat']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
    pass

In [8]:
# housing soll nun nur noch aus den trainingsdaten bestehen(kopie)
housing = strat_train_set.drop('median_house_value', axis=1).drop('income_cat', axis=1)
housing_labels = strat_train_set['median_house_value'].copy()

# 3. Transformer definieren

In [9]:
# Neu ab sklearn > 19:
# CategoricalEncoder -> übernimmt beide schritte auf einmal
# Definition of the CategoricalEncoder class, copied from PR #9151.
# Just run this cell, or copy it to your code, do not try to understand it (yet).

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.preprocessing import LabelEncoder
from scipy import sparse

class CategoricalEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, encoding='onehot', categories='auto', dtype=np.float64,
                 handle_unknown='error'):
        self.encoding = encoding
        self.categories = categories
        self.dtype = dtype
        self.handle_unknown = handle_unknown

    def fit(self, X, y=None):
        if self.encoding not in ['onehot', 'onehot-dense', 'ordinal']:
            template = ("encoding should be either 'onehot', 'onehot-dense' "
                        "or 'ordinal', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.handle_unknown not in ['error', 'ignore']:
            template = ("handle_unknown should be either 'error' or "
                        "'ignore', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.encoding == 'ordinal' and self.handle_unknown == 'ignore':
            raise ValueError("handle_unknown='ignore' is not supported for"
                             " encoding='ordinal'")

        X = check_array(X, dtype=np.object, accept_sparse='csc', copy=True)
        n_samples, n_features = X.shape

        self._label_encoders_ = [LabelEncoder() for _ in range(n_features)]

        for i in range(n_features):
            le = self._label_encoders_[i]
            Xi = X[:, i]
            if self.categories == 'auto':
                le.fit(Xi)
            else:
                valid_mask = np.in1d(Xi, self.categories[i])
                if not np.all(valid_mask):
                    if self.handle_unknown == 'error':
                        diff = np.unique(Xi[~valid_mask])
                        msg = ("Found unknown categories {0} in column {1}"
                               " during fit".format(diff, i))
                        raise ValueError(msg)
                le.classes_ = np.array(np.sort(self.categories[i]))

        self.categories_ = [le.classes_ for le in self._label_encoders_]

        return self

    def transform(self, X):
        X = check_array(X, accept_sparse='csc', dtype=np.object, copy=True)
        n_samples, n_features = X.shape
        X_int = np.zeros_like(X, dtype=np.int)
        X_mask = np.ones_like(X, dtype=np.bool)

        for i in range(n_features):
            valid_mask = np.in1d(X[:, i], self.categories_[i])

            if not np.all(valid_mask):
                if self.handle_unknown == 'error':
                    diff = np.unique(X[~valid_mask, i])
                    msg = ("Found unknown categories {0} in column {1}"
                           " during transform".format(diff, i))
                    raise ValueError(msg)
                else:
                    # Set the problematic rows to an acceptable value and
                    # continue `The rows are marked `X_mask` and will be
                    # removed later.
                    X_mask[:, i] = valid_mask
                    X[:, i][~valid_mask] = self.categories_[i][0]
            X_int[:, i] = self._label_encoders_[i].transform(X[:, i])

        if self.encoding == 'ordinal':
            return X_int.astype(self.dtype, copy=False)

        mask = X_mask.ravel()
        n_values = [cats.shape[0] for cats in self.categories_]
        n_values = np.array([0] + n_values)
        indices = np.cumsum(n_values)

        column_indices = (X_int + indices[:-1]).ravel()[mask]
        row_indices = np.repeat(np.arange(n_samples, dtype=np.int32),
                                n_features)[mask]
        data = np.ones(n_samples * n_features)[mask]

        out = sparse.csc_matrix((data, (row_indices, column_indices)),
                                shape=(n_samples, indices[-1]),
                                dtype=self.dtype).tocsr()
        if self.encoding == 'onehot-dense':
            return out.toarray()
        else:
            return out

In [10]:
rooms_ix, bedrooms_ix, population_ix, household_ix = 3,4,5,6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        rooms_per_household = X[:, population_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [12]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X[self.attribute_names].values

In [45]:
class RemoveAttribs(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_keep, add_bedrooms_per_room = True):
        self.columns_to_keep = columns_to_keep
        self.add_bedrooms_per_room = add_bedrooms_per_room
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        to_keep = self.get_index_to_keep()
        print(to_keep)
        return X[:,to_keep]
    
    def get_index_to_keep(self):
        
        all_attribs = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'rooms_per_hhold', 'pop_per_hhold', 'bedrooms_per_room', '<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN']
        
        if not self.add_bedrooms_per_room:
            all_attribs.remove('bedrooms_per_room')
        
        all_attribs = { v:k for k,v in enumerate(all_attribs)}
        
        index_to_keep = []
        
        for k, v in all_attribs.items():
            if k in self.columns_to_keep:
                index_to_keep.append(v)
                pass
            pass
                
        return index_to_keep

# 5. Pipeline definieren

In [46]:
num_attribs = list(housing.drop('ocean_proximity', axis=1))
cat_attribs = ['ocean_proximity']

In [47]:
from sklearn.preprocessing import Imputer

In [48]:
num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', Imputer(strategy='median')),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])
cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
    ('categorical_encoder', CategoricalEncoder(encoding="onehot-dense")),
])

In [49]:
from sklearn.pipeline import FeatureUnion

In [50]:
feature_union = FeatureUnion(transformer_list=[
    ('num_pipeline', num_pipeline),
    ('cat_pipeline', cat_pipeline)
])

# 6. Pipeline ausführen

In [51]:
#housing_prepared = full_pipeline.fit_transform(housing)
#housing_prepared.shape

# 7. Ganze Pipeline

In [52]:
from sklearn.pipeline import Pipeline

In [53]:
attribs_to_keep = ['median_income','INLAND','bedrooms_per_room', 'pop_per_hhold', 'longitude','latitude', 'rooms_per_hhold','housing_median_age']

full_pipeline = Pipeline([
    ('feature_union',feature_union),
    ('remove_attribs', RemoveAttribs(columns_to_keep=attribs_to_keep))
])

In [54]:
housing_prepared = full_pipeline.fit_transform(housing)

[0, 1, 2, 7, 8, 9, 10, 12]


# 8. RandomForestRegressor

In [67]:
from sklearn.ensemble import RandomForestRegressor

In [68]:
best_attribs = {'max_features': 6, 'n_estimators': 30}

random_forest = Pipeline([
    ('feature_union',feature_union),
    ('remove_attribs', RemoveAttribs(columns_to_keep=attribs_to_keep)),
    ('forest_reg',RandomForestRegressor(**best_attribs))
])

In [69]:
from sklearn.feature_selection import SelectKBest

In [70]:
random_forest.fit(housing, housing_labels)

[0, 1, 2, 7, 8, 9, 10, 12]


Pipeline(memory=None,
     steps=[('feature_union', FeatureUnion(n_jobs=1,
       transformer_list=[('num_pipeline', Pipeline(memory=None,
     steps=[('selector', DataFrameSelector(attribute_names=['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income'])), ...timators=30, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False))])

In [72]:
predicted = random_forest.predict(housing)
count = 0
for label, pred in zip(housing_labels, predicted):
    if count > 10: break
    print(label, pred)
    pass

[0, 1, 2, 7, 8, 9, 10, 12]
286600.0 273176.666667
340600.0 345806.666667
196900.0 228446.666667
46300.0 53603.3333333
254500.0 227036.666667
127900.0 135726.666667
500001.0 498870.966667
140200.0 180216.666667
95000.0 97050.0
500001.0 443120.833333
92100.0 94633.3333333
61500.0 64500.0
313000.0 321416.7
89000.0 93763.3333333
123900.0 134300.0
197400.0 197693.333333
500001.0 479804.2
63300.0 62986.6666667
107000.0 120206.666667
184200.0 186703.333333
280900.0 244543.333333
500001.0 497607.6
171300.0 173256.666667
116600.0 122186.666667
60800.0 61366.6666667
121100.0 125006.666667
270700.0 255033.333333
109900.0 107703.333333
159600.0 145920.0
500001.0 470737.366667
409100.0 380960.0
55000.0 97610.0
127500.0 142836.666667
184400.0 176066.666667
210500.0 232136.666667
277300.0 294173.333333
85000.0 85923.3333333
202100.0 180006.666667
146600.0 147350.0
326600.0 323543.366667
207300.0 200683.333333
173900.0 187916.666667
301700.0 307336.666667
274800.0 287766.666667
150800.0 163590.0
21670

272400.0 285683.4
211900.0 230700.0
266700.0 255113.333333
102700.0 113830.0
161000.0 178126.666667
73200.0 76933.3333333
57500.0 59896.6666667
193800.0 189100.0
244900.0 246560.033333
293800.0 291786.7
143800.0 122256.666667
370900.0 375990.066667
137500.0 133383.333333
239100.0 244703.333333
230000.0 243636.666667
300000.0 243086.666667
198100.0 206306.666667
70500.0 78010.0
236500.0 252653.333333
329400.0 313293.333333
162500.0 168603.333333
434500.0 441093.5
72100.0 76313.3333333
170400.0 173096.666667
159700.0 149213.333333
129700.0 131660.0
61100.0 65373.3333333
240000.0 228330.0
115200.0 117330.0
89400.0 88700.0
107200.0 108400.0
291500.0 319380.0
150000.0 150286.666667
208700.0 212623.333333
105300.0 109370.0
206300.0 208600.0
326000.0 347383.366667
88300.0 94803.3333333
156800.0 160293.333333
335000.0 321956.7
247200.0 289016.666667
285600.0 303460.0
122200.0 129496.666667
110000.0 136393.333333
129300.0 139880.0
223000.0 217133.333333
170700.0 166030.0
162500.0 161836.666667


116200.0 114016.666667
219000.0 214973.333333
217800.0 225216.7
500001.0 500001.0
136400.0 143130.0
235600.0 234303.333333
198500.0 182563.333333
73400.0 75853.3333333
77300.0 72430.0
76300.0 84110.0
396000.0 354413.333333
117900.0 115636.666667
239100.0 250923.333333
462700.0 402623.333333
61600.0 68666.6666667
335300.0 341593.4
261300.0 271196.666667
234000.0 238243.333333
127100.0 129176.666667
186300.0 186450.0
236200.0 248496.666667
240900.0 249473.333333
225000.0 201153.333333
271400.0 279783.333333
191100.0 197270.0
219200.0 237526.666667
192300.0 192976.666667
348500.0 360770.033333
207500.0 227010.066667
70000.0 73313.3333333
251900.0 260150.0
381900.0 365443.366667
155000.0 173786.666667
174500.0 164616.666667
220500.0 215890.0
170300.0 168786.666667
418300.0 407553.366667
69700.0 72153.3333333
187500.0 184366.666667
110100.0 128896.666667
226800.0 251813.333333
274200.0 250636.666667
263600.0 253356.666667
233300.0 247743.366667
120000.0 116026.666667
99800.0 96233.3333333
1

413900.0 393923.333333
123300.0 131210.0
500001.0 489607.533333
41700.0 45986.6666667
195100.0 212163.333333
296300.0 282450.0
141400.0 146810.0
222500.0 235406.666667
136400.0 137503.333333
186000.0 189583.333333
125000.0 118653.333333
137500.0 186390.033333
179200.0 184346.666667
112700.0 126710.0
48100.0 62683.3333333
144100.0 146470.0
163200.0 158876.666667
57900.0 58343.3333333
203400.0 193843.333333
483300.0 461073.333333
95500.0 99390.0
450000.0 449316.833333
263900.0 295060.0
146300.0 152906.666667
53000.0 56636.6666667
215000.0 185713.333333
179500.0 172136.666667
328000.0 327813.333333
162500.0 161576.666667
260500.0 265256.666667
232500.0 267890.0
209800.0 225633.333333
156900.0 166020.0
135300.0 142070.0
251000.0 245303.333333
87500.0 81196.6666667
95300.0 103593.333333
267500.0 301046.7
367100.0 371116.666667
186300.0 201866.666667
50000.0 48983.3333333
160400.0 146656.666667
340700.0 339333.333333
128700.0 140093.333333
177200.0 181613.333333
152800.0 172206.666667
160200

190100.0 197043.333333
76800.0 76473.3333333
262500.0 251463.333333
350000.0 304273.333333
156500.0 162550.0
131300.0 136983.333333
313600.0 338416.666667
144600.0 145200.0
100000.0 93740.0
250300.0 249843.333333
87500.0 89893.3333333
113600.0 119970.0
162500.0 198053.333333
193500.0 202463.333333
500001.0 428977.333333
163000.0 164060.0
245100.0 261593.333333
146000.0 170453.333333
153100.0 166376.666667
147600.0 178036.666667
361700.0 333033.333333
163100.0 160263.333333
500001.0 483447.566667
376000.0 369073.466667
105300.0 103786.666667
185600.0 208883.333333
293200.0 327780.133333
139600.0 138133.333333
238000.0 233513.333333
115400.0 122336.666667
176800.0 172996.666667
126900.0 124890.0
178100.0 180713.333333
153000.0 154803.333333
90300.0 88813.3333333
340100.0 338526.666667
193800.0 210663.333333
62000.0 69376.6666667
62100.0 59793.3333333
137500.0 146083.333333
59100.0 62273.3333333
285300.0 295496.666667
229300.0 251396.666667
500001.0 500001.0
310600.0 290796.666667
500001.

...

178200.0 175450.0
57500.0 60856.6666667
464600.0 391036.7
291200.0 297583.333333
218800.0 202843.333333
284900.0 290293.333333
223800.0 223660.0
265400.0 315546.733333
106300.0 107263.333333
100000.0 107933.333333
221600.0 223196.666667
169600.0 165100.0
67900.0 79053.3333333
118300.0 122576.666667
425100.0 452566.966667
285400.0 314400.066667
180000.0 182233.333333
55100.0 59786.6666667
253900.0 249700.0
107700.0 115353.333333
159400.0 175150.0
145200.0 147556.666667
228100.0 218260.0
213000.0 216586.666667
220300.0 217636.666667
357200.0 403686.933333
191900.0 195756.666667
292500.0 287780.0
178600.0 166316.666667
457100.0 461540.133333
142600.0 146233.333333
226500.0 224600.0
240800.0 192803.333333
45600.0 47796.6666667
138300.0 144633.333333
119800.0 123740.0
120100.0 115280.0
153300.0 160740.0
227300.0 244726.666667
204200.0 195560.0
142500.0 125853.333333
262500.0 245553.333333
500001.0 466450.766667
134100.0 129463.333333
277700.0 272966.666667
96100.0 103240.0
485700.0 403926.6

342700.0 346080.0
350000.0 387873.6
94600.0 98526.6666667
115900.0 116803.333333
69700.0 74736.6666667
267300.0 256386.666667
134500.0 157453.333333
104000.0 103453.333333
326500.0 241963.333333
164000.0 170723.333333
171300.0 195683.333333
201700.0 175606.666667
53300.0 56573.3333333
228700.0 267233.366667
226900.0 219673.333333
75000.0 79900.0
361500.0 336670.0
187500.0 189980.0
316700.0 307550.0
295600.0 276896.666667
141700.0 167070.0
161100.0 147053.333333
356100.0 358340.066667
158500.0 155150.0
157100.0 150323.333333
187200.0 182816.666667
236400.0 287380.133333
268800.0 220676.666667
145500.0 139890.0
87500.0 101470.0
233300.0 223206.666667
186000.0 196696.666667
86900.0 83173.3333333
247300.0 259640.0
126900.0 143753.333333
152700.0 149656.666667
236800.0 234136.666667
500001.0 500001.0
65200.0 66633.3333333
401700.0 428977.0
92800.0 99856.6666667
80400.0 83920.0
120100.0 167006.666667
140600.0 241546.8
371800.0 397610.033333
183800.0 211483.333333
102000.0 100406.666667
50000

184100.0 181193.333333
285500.0 284330.0
230600.0 238980.0
73600.0 77470.0
174300.0 175756.666667
240700.0 278980.033333
500001.0 455780.6
151900.0 174453.333333
113500.0 113010.0
169900.0 168590.0
196200.0 193750.0
159800.0 157020.0
169300.0 165823.333333
173200.0 172760.0
258600.0 248300.0
500001.0 414054.0
67500.0 69506.6666667
339500.0 332066.7
155600.0 160473.333333
72800.0 77693.3333333
362800.0 369420.033333
164000.0 165033.333333
87500.0 101730.0
284100.0 273360.0
265900.0 265863.333333
87800.0 87886.6666667
247500.0 228883.333333
254600.0 257463.333333
402600.0 358850.033333
216700.0 235356.666667
123500.0 127826.666667
194400.0 215243.366667
258600.0 258370.0
488900.0 446036.733333
120800.0 129896.666667
127600.0 149840.0
500001.0 499437.633333
59200.0 70263.3333333
92400.0 95390.0
248700.0 236720.0
77400.0 90883.3333333
297200.0 289690.0
186800.0 216833.333333
398800.0 396606.7
274200.0 255886.666667
500000.0 442676.7
160300.0 162843.333333
186600.0 186980.0
152500.0 150023.

171600.0 164963.333333
117600.0 126680.0
159200.0 164500.0
275000.0 259383.333333
240000.0 230433.333333
76600.0 73690.0
157400.0 155803.333333
500001.0 499954.3
168200.0 173433.333333
500001.0 499070.966667
94000.0 97963.3333333
90000.0 90280.0
73500.0 96216.6666667
100000.0 112176.666667
191100.0 182346.666667
187500.0 204423.333333
311100.0 290023.333333
116100.0 131106.666667
241700.0 231716.666667
195600.0 200173.333333
241600.0 250480.0
404800.0 372476.666667
289400.0 281546.666667
220500.0 220600.0
175000.0 177036.666667
121400.0 128490.0
60800.0 66586.6666667
100000.0 99313.3333333
307400.0 317493.4
150100.0 156636.666667
156100.0 155093.333333
156900.0 136330.0
183800.0 193496.666667
55800.0 54470.0
123800.0 147243.333333
335000.0 303746.7
217500.0 228520.0
185400.0 203630.0
472100.0 474120.2
353100.0 344246.666667
500001.0 497367.6
409800.0 397923.333333
57500.0 66083.3333333
253000.0 265263.333333
107800.0 113293.333333
155000.0 165423.333333
182900.0 184956.666667
239900.0 

107500.0 105716.666667
165100.0 166733.333333
384200.0 376623.333333
122500.0 172050.066667
350000.0 299943.333333
172800.0 172716.666667
500001.0 500001.0
146600.0 143896.666667
81300.0 79190.0
247600.0 220330.0
164100.0 156336.666667
254900.0 277340.0
185700.0 194486.666667
126800.0 122360.0
114200.0 113913.333333
101800.0 107866.666667
265600.0 271913.333333
88900.0 92110.0
225000.0 208570.0
123500.0 126256.666667
500001.0 492967.533333
321600.0 325710.0
266000.0 271086.666667
346700.0 312996.666667
190200.0 190573.333333
148800.0 142163.333333
214800.0 216900.0
174300.0 166586.666667
240200.0 247746.666667
113000.0 110363.333333
97800.0 108176.666667
225900.0 203963.333333
500001.0 465594.033333
